# ElasticSearch 

Image: https://hub.docker.com/r/bitnami/elasticsearch#!&_escaped_fragment_=

Ports: 9200, 9201, 9300
Mount: -v /path/to/elasticsearch-data-persistence:/bitnami/elasticsearch/data

curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'

curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}


http://localhost:9200/?pretty

Array mapping: https://stackoverflow.com/questions/67581594/elasticsearch-mapping-for-array

Habr guide: https://habr.com/ru/articles/280488/'

# Kibana

Image: https://github.com/bitnami/containers/tree/main/bitnami/kibana


docker network create kibana_network

docker run -d -p 9200:9200 --name elasticsearch --net=kibana_network bitnami/elasticsearch

docker run -d -p 5601:5601 --name kibana --net=kibana_network  -e KIBANA_ELASTICSEARCH_URL=elasticsearch   bitnami/kiba


docker run -v /path/to/kibana-persistence:/bitnami/kibana bitnami/kibana:latest
na

# Conatiners

## elasticsearch

```bash
docker run -d -p 9200:9200 -v "C:\Users\sergey.astakhov\Desktop\volumes\elastic_search_volume":/bitnami/elasticsearch/data --name elasticsearch --net=kibana_network bitnami/elasticsearch
```

## kibana

```bash
docker run -d -p 5601:5601 -v C:\Users\sergey.astakhov\Desktop\volumes\kibana_volume:/bitnami/kibana --name kibana --net=kibana_network -e KIBANA_ELASTICSEARCH_URL=elasticsearch bitnami/kibana
```

All orders: http://localhost:9200/order/_search?pretty

All masters: http://localhost:9200/master/_search?pretty

Dashboard: http://localhost:5601/app/dashboards#/view/bdcf6af3-3542-4936-94fd-b0f04e3dcc86?_g=(filters:!(),refreshInterval:(pause:!t,value:60000),time:(from:'2023-12-15T00:00:00.000Z',to:'2024-02-21T00:00:00.000Z'))

In [156]:
%%html
<iframe src="http://localhost:5601/app/dashboards#/view/bdcf6af3-3542-4936-94fd-b0f04e3dcc86?embed=true&_g=(refreshInterval:(pause:!t,value:60000),time:(from:'2023-12-15T00:00:00.000Z',to:'2024-02-21T00:00:00.000Z'))&_a=()" height="600" width="800"></iframe>

In [2]:
import requests

In [3]:
r = requests.get('http://127.0.0.1:5601/api/kibana/dashboards/export?dashboard=bdcf6af3-3542-4936-94fd-b0f04e3dcc86')

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5601): Max retries exceeded with url: /api/kibana/dashboards/export?dashboard=bdcf6af3-3542-4936-94fd-b0f04e3dcc86 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E762908B50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [164]:
import json
with open('../dashboard.json', 'w') as fp:
    json.dump(r.json(), fp)

In [4]:
r = requests.put('http://localhost:9200/_cluster/settings', json={ "transient": { "cluster.routing.allocation.disk.threshold_enabled": False } })

In [5]:
r.json()

{'acknowledged': True,
 'persistent': {},
 'transient': {'cluster': {'routing': {'allocation': {'disk': {'threshold_enabled': 'false'}}}}}}

In [6]:
r = requests.put('http://localhost:9200/_all/_settings', json={"index.blocks.read_only_allow_delete": None})

In [7]:
r.json()

{'acknowledged': True}

In [8]:
import json
from elasticsearch import Elasticsearch

In [9]:
client = Elasticsearch([{"host": "127.0.0.1", "scheme": "http", "port": 9200}])

client.ping()

True

# Index 1 - orders

In [166]:
indexName = "order"
client.indices.delete(index=indexName)
# client.indices.create(index=indexName)

ObjectApiResponse({'acknowledged': True})

In [167]:
analyzer_settings = {
  "settings": {
    "analysis": {
      "filter": {
        "ru_stop": {
          "type": "stop",
          "stopwords": "_russian_"
        },
        "ru_stemmer": {
          "type": "stemmer",
          "language": "russian"
        }
      },
      "analyzer": {
        "custom_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "ru_stop",
            "ru_stemmer"
          ]
        }
      }
    }
  }
}

In [168]:
mappings_order = {
"mappings" : {
  "properties" : {
    "order_customer_desc" : {
      "type" : "text",
      "analyzer": "custom_analyzer"
    },
    "order_customer_id" : {
      "type" : "long"
    },
    "order_date" : {
      "type" : "date"
    },
    "order_details_desc" : {
      "type" : "text",
      "analyzer": "custom_analyzer"
    },
    "order_due_date" : {
      "type" : "date"
    },
    "order_fact_completion_date" : {
      "type" : "date"
    },
    "order_id" : {
      "type" : "long"
    },
    "order_master_id" : {
      "type" : "long"
    },
    "order_parts" : {
      "type" : "text",
      "fields" : {
        "keyword" : {
          "type" : "keyword"
        }
      }
    },
    "order_price" : {
      "type" : "float"
    },
    "repair_types" : {
      "type" : "text",
      "fields" : {
        "keyword" : {
          "type" : "keyword"
        }
      }
    }
  }
}
}


client.indices.create(index=indexName, 
                            # doc_type="diseases", 
                           # include_type_name="true", 
                           body={**analyzer_settings
                                 , **mappings_order
                                }
                          )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'order'})

In [169]:
with open('../orders.json', 'r') as f:
    dataStore = json.load(f)

for data in dataStore:
    try:
        client.index(
            index=data["index"],
            # doc_type=data["doc_type"],
            id=data["id"],
            body=data["body"]
        )
    except Exception as e:
        print(e, end="")
print("index 'order' data inserted")

index 'order' data inserted


# Query 1 - orders

In [145]:
indexName = "order"
searchSize = 9999

searchBody = {
  "size": searchSize,
  # "query" : {
  #   "match_all" : {}
  # }
  "_source": True,
  # "query": {},
  "aggs": {
    "over_months": {
      "date_histogram": {
        "field": "order_date",
        "calendar_interval": "month",
        "format": "yyyy-MM-dd" # control the output format
      },
        "aggs":{
            "over_parts": {
              "terms": {
                "field": "order_parts.keyword"
                # "fielddata": True
              },
            }
        }
    }
  }
}

result = client.search(index=indexName, body=searchBody)

In [146]:
# result

In [147]:
result['aggregations']['over_months']['buckets']

[{'key_as_string': '2023-12-01',
  'key': 1701388800000,
  'doc_count': 20,
  'over_parts': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'аккумулятор', 'doc_count': 9},
    {'key': 'дисплей', 'doc_count': 8},
    {'key': 'корпус', 'doc_count': 8},
    {'key': 'usb-разъем', 'doc_count': 6}]}},
 {'key_as_string': '2024-01-01',
  'key': 1704067200000,
  'doc_count': 42,
  'over_parts': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'usb-разъем', 'doc_count': 18},
    {'key': 'аккумулятор', 'doc_count': 17},
    {'key': 'корпус', 'doc_count': 17},
    {'key': 'дисплей', 'doc_count': 15}]}},
 {'key_as_string': '2024-02-01',
  'key': 1706745600000,
  'doc_count': 38,
  'over_parts': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'дисплей', 'doc_count': 19},
    {'key': 'аккумулятор', 'doc_count': 18},
    {'key': 'корпус', 'doc_count': 18},
    {'key': 'usb-разъем', 'doc_

In [148]:
# result['hits']

# Index 2 - masters

In [149]:
indexName = "master"
client.indices.delete(index=indexName)
# client.indices.create(index=indexName)

ObjectApiResponse({'acknowledged': True})

In [150]:
mappings_master = {
"mappings" : {
      "properties" : {
        "master_desc" : {
          "type" : "text",
          "analyzer": "custom_analyzer"
        },
        "master_feedbacks" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "master_id" : {
          "type" : "long"
        }
      }
}
}

client.indices.create(index=indexName, 
                            # doc_type="diseases", 
                           # include_type_name="true", 
                           body={**analyzer_settings
                                 , **mappings_master
                                }
                          )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'master'})

In [151]:
with open('../masters.json', 'r') as f:
    dataStore = json.load(f)

for data in dataStore:
    try:
        client.index(
            index=data["index"],
            # doc_type=data["doc_type"],
            id=data["id"],
            body=data["body"]
        )
    except Exception as e:
        print(e, end="")
print("index 'master' data inserted")

index 'master' data inserted


# Query 2 - masters

In [152]:
indexName = "master"
searchFrom = 0
searchSize = 5

searchBody = {
  "_source": True, #["master_desc"], # True - all data
  "query": {
    "match": {
      "master_desc": "стаж"
    }
  },
  "size": searchSize
}

result = client.search(index=indexName, body=searchBody)

In [153]:
# result

In [154]:
result['hits']

{'total': {'value': 12, 'relation': 'eq'},
 'max_score': 0.49623758,
 'hits': [{'_index': 'master',
   '_id': '618529',
   '_score': 0.49623758,
   '_source': {'master_id': 618529,
    'master_desc': 'Аполлон Харитонович Афанасьев, Стаж Работы: 5 Л./Г..',
    'master_feedbacks': ['медлительный, аккуратный.',
     'ворчливый, медлительный.']}},
  {'_index': 'master',
   '_id': '791839',
   '_score': 0.49623758,
   '_source': {'master_id': 791839,
    'master_desc': 'Август Дорофеевич Кошелев, Стаж Работы: 11 Л./Г..',
    'master_feedbacks': ['медлительный, ворчливый.']}},
  {'_index': 'master',
   '_id': '96520',
   '_score': 0.49623758,
   '_source': {'master_id': 96520,
    'master_desc': 'Давыдова Александра Харитоновна, Стаж Работы: 12 Л./Г..',
    'master_feedbacks': ['аккуратный, медлительный.',
     'медлительный, аккуратный.',
     'ворчливый, медлительный.']}},
  {'_index': 'master',
   '_id': '377918',
   '_score': 0.49623758,
   '_source': {'master_id': 377918,
    'master_de